# PSI3471 - Entrega 01 
### Fernando Zolubas Preto - NUSP: 10694192
### Vinícius Melo de Souza - NUSP: 10772272
# Regressão Linear 

# Tarefas

- 0) Intuir combinação de variaveis ótima

- 1) Criar Função que dada uma tabela de dados retorna a regressão para preço

- 2) Dummyzar variaveis qualitativas

- 3) Criar Variaveis Novas

- 4) Chamar Função para todas as combinações de variaveis, salvar predição da base testes e escolher melhor.

- 5) Comparar resultado da intuição versus melhor resultado numerico obtido

Neste exercício queremos prever o preço de carros do _"Used Cars Dataset"_ do site `Craiglist.org`.

No caso, os dados já foram limpos e filtrados para facilitar o entendimento e o andamento do exercício, para conseguirmos dar foco na utilização da técnica dos mínimos quadrados. Os dados limpos estão em um arquivo CSV que podemos puxar do site da disciplina. Este arquivo será importado através da biblioteca "Panda" do python que foi instalada no computador previamente.

In [27]:
import pandas as pd
import numpy as np
cars_data = pd.read_csv("vehicles_cleaned_train.csv")

Com esses dados, queremos criar um _"DataFrame"_ que estaremos chamando de __cars_data__, que irá conter esses dados do arquivo CSV. Neste _DataFrame_ estaremos gerando os arrays NumPy para calcular os parâmetros do modelo de regressão linear.

O banco de dados limpo possui os seguintes dados(exemplo):

In [28]:
cars_data.head()

,price,year,condition,cylinders,fuel,odometer,transmission,size,type
0,10400,2011,excellent,4 cylinders,gas,81300,automatic,mid-size,sedan
1,6900,2007,excellent,6 cylinders,gas,79000,automatic,full-size,sedan
2,18900,2018,like new,4 cylinders,gas,5000,automatic,full-size,sedan
3,6000,2010,excellent,4 cylinders,gas,97600,automatic,mid-size,hatchback
4,19995,2013,good,6 cylinders,gas,95782,automatic,mid-size,sedan


Temos essas 9 colunas que se referem à:

- __Price__: O preço do carro. É o dado que desejamos obter com o modelo.
- __Year__: Ano do carro.
- __Condition__: Variável categórica que indica a condição do carro. Pode ter os valores _good_, _fair_, _excellent_, _like new_, _salvage_, ou _new_.
- __Cylinders__: Variável categórica que indica o número de cilindros do motor. Pode ter os valores _4 cylinders_ ou _6 cylinders_.
- __Fuel__: Variável categórica que indica o combustível do carro. Pode ter os valores _gas_ ou _diesel_.
- __Odometer__: Valor registrado no odômetro, em milhas.
- __Transmission__: Variável categórica que indica o tipo de transmissão. Pode ter os valores _automatic_ ou _manual_.
- __Size__: Variável categórica que indica o tamanho do carro. Pode ter os valores _compact_, _mid-size_, _sub-compact_ ou _full-size_.
- __Type__:	Variável categórica que indica o tipo do carro. Pode ter os valores _sedan_, _coupe_, _wagon_, ou _hatchback_.


Para usar os dados categóricos de forma numérica iremos estar utilizando variáveis _dummy_.

E por fim, para obter o vetor "wo" vamos seguir os seguintes passos.

1. Selecionar o conjunto de variáveis originais que vocễ vai utilizar no modelo.
2. Substituir cada variável categórica de sua seleção por um conjunto de variáveis dummy, conforme descrito anteriormente.
3. Transformar as variáveis originais de sua seleção e / ou incluir combinações, caso julgue necessário;
4. A partir de sua seleção de dados, obter a matriz X e o vetor d, que podem ser representados como arrays do NumPy.
5. Usando a matriz X e o vetor d, calcular o vetor wo e o erro quadrático médio conforme mostrado na aula.

# Nossa solução

1. Função de regressão linear

In [29]:
def RL_reta(data):
    """
    Função que implementa a regressão linear multipla para prever o preço de carros.

    dados é um dataframe com as colunas:
    - price: O preço do carro.
    - year: O ano do carro.
    - condition: Variável categórica que indica a condição do carro.
    - cylinders: Variável categórica que indica o número de cilindros do motor.
    - fuel: Variável categórica que indica o combustível do carro.
    - odometer: Valor registrado no odômetro, em milhas.
    - transmission: Variável categórica que indica o tipo de transmissão.
    - size: Variável categórica que indica o tamanho do carro.
    - type: Variável categórica que indica o tipo do carro.

    """

    # Extrai as colunas relevantes dos dados de entrada
    x = data['year'].values.reshape(-1, 1)
    d = data['price'].values.reshape(-1, 1)

    # Cria as variáveis dummy para as variáveis categóricas
    condition_dummy = pd.get_dummies(data['condition'])
    cylinders_dummy = pd.get_dummies(data['cylinders'])
    fuel_dummy = pd.get_dummies(data['fuel'])
    odometer_dummy = pd.get_dummies(data['odometer'])
    transmission_dummy = pd.get_dummies(data['transmission'])
    size_dummy = pd.get_dummies(data['size'])
    type_dummy = pd.get_dummies(data['type'])

    # Concatena as variáveis dummy com os dados de entrada originais
    x = np.hstack([x, condition_dummy, cylinders_dummy, fuel_dummy, odometer_dummy, transmission_dummy, size_dummy, type_dummy])

    # N representa o número de dados para o cálculo da RL
    N = d.shape[0]

    # X é a matriz dos dados
    X = np.hstack([np.ones((N, 1)), x])

    # R é o produto das matrizes X.T e X
    R = X.T @ X

    # o vetor p é dado pelo produto entre a matriz X.T e o vetor d
    p = X.T @ d

    # Calcule a solução wo e o erro e
    wo = np.linalg.solve(R, p)
    e = d - X @ wo

    return wo, e

In [37]:
wo, e = RL_reta(cars_data)

print(e)

[[ 2.53953133e+03]
 [-1.61896431e+03]
 [-1.85653000e+03]
 ...
 [-2.13724780e+03]
 [ 2.21189111e-09]
 [-3.74706274e+03]]


In [57]:
cars_data_teste = pd.read_csv("vehicles_cleaned_train.csv")
cars_data_teste.head()

,price,year,condition,cylinders,fuel,odometer,transmission,size,type
0,10400,2011,excellent,4 cylinders,gas,81300,automatic,mid-size,sedan
1,6900,2007,excellent,6 cylinders,gas,79000,automatic,full-size,sedan
2,18900,2018,like new,4 cylinders,gas,5000,automatic,full-size,sedan
3,6000,2010,excellent,4 cylinders,gas,97600,automatic,mid-size,hatchback
4,19995,2013,good,6 cylinders,gas,95782,automatic,mid-size,sedan


In [64]:
# criar variáveis dummy para as variáveis categóricas
condition_dummy = pd.get_dummies(cars_data_teste['condition'])
cylinders_dummy = pd.get_dummies(cars_data_teste['cylinders'])
fuel_dummy = pd.get_dummies(cars_data_teste['fuel'])
odometer_dummy = pd.get_dummies(cars_data_teste['odometer'])
transmission_dummy = pd.get_dummies(cars_data_teste['transmission'])
size_dummy = pd.get_dummies(cars_data_teste['size'])
type_dummy = pd.get_dummies(cars_data_teste['type'])

In [65]:
xteste = cars_data_teste['year'].values.reshape(-1, 1)
xteste = np.hstack([xteste, condition_dummy, cylinders_dummy, fuel_dummy, odometer_dummy, transmission_dummy, size_dummy, type_dummy])
dteste = cars_data_teste['price'].values.reshape(-1, 1)
# N representa o número de dados para o cálculo da RL
Nteste = dteste.shape[0]

# X é a matriz dos dados
XTeste = np.hstack([np.ones((Nteste, 1)), xteste])
dTeste = cars_data_teste['price'].values.reshape(-1, 1)

print(dTeste.shape)
print(XTeste.shape)
print(wo.shape)

(8338, 1)
(8338, 4039)
(4043, 1)


In [66]:
# para cada linha do conjunto de teste
for i in range(len(cars_data_teste)):
    # extrai as features da linha i
    xTeste_i = XTeste[i, :]
    # calcula o valor predito pelo modelo
    y_pred_i = xTeste_i @ wo
    # calcula o erro em relação ao valor real
    e_i = dTeste[i] - y_pred_i
    # imprime o valor predito, o valor real e o erro
    print(f"Valor predito: {y_pred_i}, Valor real: {dTeste[i]}, Erro: {e_i}")

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 4043 is different from 4039)

2. Nossas variáveis originais:

- DEFINIR QUAIS VARIAVEIS VAMOS QUERER USAR

3. Substituindo as variáveis categóricas por variáveis dummy:

4. Transformar variáveis:

- Acho interessante trocar o "Year" por "Idade" do carro, acho que é mais interessante encontrar a relação entre idade do carro e o preço.
- Acho interessante entender a relação entre "Quilometragem" pela "Idade" (divisão).

E para criar elas no DataFrame:

4. Obter matriz X e o vetor d